In [1]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact

def fourierPulse(pulselength):
    plt.clf()
    freq = 6.0
    def f(t):
        return np.sin(2*np.pi* freq*t)
    nframes = 1000
    timeDuration = pulselength
    t = np.linspace(0.0, timeDuration, nframes)

    sampRate = float(nframes)/float(timeDuration)

    fig, (ax1,ax2, ax3) = plt.subplots(1,3)
    fig.set_size_inches(16, 5)
    ft = f(t)
    ax1.plot(t, ft)
    ax1.set_xlabel('t', fontsize = 15)
    ax1.set_ylabel('f(t)', fontsize = 15)
    ax1.set_xlim(0,6)
    ax1.set_ylim(-2,2)

    freqArray = np.fft.fftfreq(6000, 1.0/sampRate)
    fft= np.fft.fft(ft, len(freqArray))

    ax2.plot(freqArray, fft.real)
    ax2.set_xlim(-freq-10.0, freq+10.0)
    ax3.plot(freqArray, fft.imag)
    ax3.set_xlim(-freq-10.0, freq+10.0)

    plt.show()
    
interact(fourierPulse, pulselength= (0.5,5,0.1)) 

<function __main__.fourierPulse>

In [13]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact

def fourierPulse(timeLength):
    plt.clf()
    freq1 = 6.0
    freq2 = 6.5
    def f(t):
        return np.exp(-t)*(np.sin(2*np.pi* freq1*t)+ np.sin(2*np.pi* freq2*t))
    nframes = 1000
    timeDuration = timeLength
    t = np.linspace(0.0, timeDuration, nframes)

    sampRate = float(nframes)/float(timeDuration)

    fig, (ax1,ax2) = plt.subplots(1,2)
    fig.set_size_inches(16, 5)
    ft = f(t)
    ax1.plot(t, ft)
    ax1.set_xlabel('t', fontsize = 15)
    ax1.set_ylabel('f(t)', fontsize = 15)
    ax1.set_xlim(0,6)
    ax1.set_ylim(-2,2)

    freqArray = np.fft.fftfreq(6000, 1.0/sampRate)
    fft= np.fft.fft(ft, len(freqArray))

    ax2.plot(freqArray, np.sqrt(fft.real**2+ fft.imag**2))
    ax2.set_xlim(0, freq2+10.0)

    plt.show()
    
interact(fourierPulse, timeLength= (0.5,5,0.1)) 

<function __main__.fourierPulse>

In [2]:
import scipy.io.wavfile
import wave
import matplotlib.pyplot as plt
import numpy as np

def loadData(wavFile):
    w = scipy.io.wavfile.read(wavFile)
    sampRate = w[0]
    data = np.array(w[1][:,0])
    return sampRate, data


def plotData(data, sampRate):
    totTime = float(len(data))/float(sampRate) 
    
    t = np.linspace(0.0, totTime,len(data))
    plt.plot(t, data)
    plt.show()
    
def fftPlot(fft, freq):
    #plt.plot(freq, fft.real)
    #plt.plot(freq, fft.imag)
    ftAbs = np.sqrt(fft.real**2+fft.imag**2)
    plt.plot(freq, ftAbs)
    plt.show()
    
    
def truncateData(sampRate, data, timeWindow):
    fpw = int(np.floor(sampRate* timeWindow))
    totFrames = len(data)
    nWindows = totFrames/fpw
    newTotFrames = nWindows *fpw
    newDat = data[0:newTotFrames]
    return newDat
    
def fft(data,freqLength):

    fft = np.fft.fft(data, freqLength)
    return fft

def windowData(data, begTime, endTime, sampRate):
    begFrame = int(begTime*sampRate)
    endFrame = int(endTime*sampRate)
    dataSamp = data[begFrame:endFrame]
    return dataSamp

def avgSquare(datasamp):
    np.sum(datasamp.astype(float), datasamp.astype(float))/float(len(datasamp))
    
def constructFftWindow(wavFile, timeWindow, beginTime = None, endTime = None):
    sRate, data = loadData(wavFile)
    print len(data)
    
    if beginTime !=None:
        begFrame = int(sRate*beginTime)
    else:
        beginTime = 0.0
        begFrame = 0
        
    if endTime != None:
        endFrame = int(sRate*endTime)
    else:
        endFrame = len(data)
    data = data[begFrame:endFrame]
        
    data = truncateData(sRate, data, timeWindow)
    
    nframes = (len(data))
    print len(data)

    # windows per frame
    fpw = int(sRate* timeWindow)
    nWind = nframes/fpw
    
    endTime = beginTime+ float(nframes)/float(sRate)
    #tArray = np.zeros((nWind, len(data)))
    #fArray = np.zeros((nWind, len(data)))

    freq = np.fft.fftfreq(10000, 1.0/(np.pi*2*sRate))
    lnFreq = np.log(freq[1:len(freq)/4])
    ftArray = np.zeros((len(freq)/4-1, nWind))
    t = np.linspace(beginTime, endTime, nWind)
    print beginTime, endTime
    for i in range(nWind): 
        bTime = i*timeWindow 
        eTime = (i+1)*timeWindow
        windData = windowData(data, bTime, eTime, sRate)
        #print np.sum(windData)
        ft = fft(windData, len(freq))
        
        ftAbs =  np.sqrt((ft.real[1:len(freq)/4])**2 + (ft.imag[1:len(freq)/4])**2)
        #print np.sum(ftAbs)
        #tArray[i, :] = np.ones(len(data))* begTime
        #fArray[i, : ] = freq
        ftArray[:, i] = ftAbs
    print np.shape(ftArray)
    print np.shape(lnFreq)
    print np.shape(t)

    plt.subplot(1,3, 1)
    

    t1 = np.linspace(beginTime, endTime, nframes)
    plt.plot(t1,data)
    
    
    plt.subplot(1,3,2)
    cmap= plt.get('Blues')
    plt.axis([t.min(), t.max(), lnFreq.min(), lnFreq.max()])
    plt.pcolor(t,lnFreq, ftArray, cmap=cmap, vmin=ftArray.min(), vmax = ftArray.max())
    #plt.pcolormesh(tArray,fArray,ftArray)
    plt.colorbar()


    fullFt = fft(data,len(freq))
    ift= np.fft.ifft(fullFt, n = len(data))
    time = np.linspace(0.0, (endTime-beginTime), len(data))
    plt.subplot(1,3,3)
    plt.plot(time,ift)
    
    plt.show()
constructFftWindow('ali.wav', 0.05)
#sRate, data = loadData("emeril.wav")


#dataNew = truncateData(sRate, data, 0.05)

#ft, freq = fft(dataNew, sRate)
#fftPlot(ft,freq)
#plt.plot(timeSmall, dataSmall)
#t = np.linspace(0, totTime, len(data))
#plt.plot(t,data)
#freq = np.fft.fftfreq(len(dataSmall), 1.0/sampRate)
#fft = np.fft.fft(dataSmall, len(freq))
#plt.plot(freq, fft.real)
#plt.plot(freq, fft.imag)

#plt.plot(freq,fft.real)
#plt.plot(freq[0:100000],fft.imag[0:100000])
#plt.show()

IndexError: too many indices for array